*langchain*

In [8]:
!pip install requests-cache


In [9]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de empresas
empresas = [
    "Silo Pharma",
    "optimi health",
    "Tilray",
    "Numinus Wellness",
    "MindMed",
    "Compass Pathways",
    "Field Trip Health",
    "Cybin",
    "ATAI Life Sciences",
    "Seelos Therapeutics",
    "Mind Cure Health",
    "Tryp Therapeutics",
    "Revive Therapeutics",
    "Red Light Holland",
    "Hollister Biosciences",
    "Mydecine Innovations Group",
    "Numinus Wellness",
    "cronosgroup",
    "Ascend Wellness Holdings",
    "Verano Holdings",
    "Columbia Care",
    "Green Thumb Industries",
    "Curaleaf Holdings",
    "Trulieve Cannabis",
    "SNDL Inc",

    "Altitude Investment Management",
    "Alinga Capital",
    "Aurora Cannabis",
    "acreageholdings",
    "4 Front Ventures",
    "Barenberg Capital Partners",
    "Bayline Capital Partners",
    "Big Rock",
    "Bullpen Capital",
    "cannaangelsllc.com",
    "cannahedge.com",
    "Canalis Capital",
    "Cannabis Capital",
    "Canopy Rivers",
    "canopyboulder",
    "Charlottes Web Holdings",
    "casaverdecapital",
    "canopygrowth",
    "Compass Cannabis Clinic",
    "Constellation Brands",
    "Craft Ventures",
    "Cusp Capital",
    "Capital 8 Group",
    "First Trust Portafolio",
    "FundRx",
    "KES7 Capital",
    "Krauter Group",
    "KushCo Holdings",
    "LaMarch Capital",
    "Lerer Hippeau Ventures",
    "Mainline Investment Partners",
    "Mazakali",
    "Medmen",
    "Mentor Capital",
    "Merida Capital Partners",
    "Miracle Mile Advisors",
    "Montegra Capital",
    "muse capital",
    "Navy Capital",
    "Nosara Capital Growthinc",
    "Phyto Partners",
    "Privateer Holdings",
    "Rose Capital",
    "Salveo Capital",
    "Green Acre Capital",
    "Green Growth Investments",
    "Green Leaf Investments",
    "Initiative Capital",
    "Supreme Cannabis",
    "The Arcview Group",
    "Tress Capital",
    "High Tide",
    "Crowe",
    "Flower Hire",
    "CHICAGO ATLANTIC GROUP",
  
  
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=20)).strftime('%Y-%m-%d')

news_data = {}

for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    params = {
        'q': empresa,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        news_data[empresa] = []
        for article in articles:
            title = article['title']
            link = article['url']
            snippet = article['description']
            published_at = article['publishedAt']
            news_data[empresa].append(
                {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        news_data[empresa] = f"Error fetching data: {str(e)}"
        print(f"Error al buscar noticias para {empresa}: {str(e)}")

# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Silo Pharma
Encontradas 14 noticias para Silo Pharma
Buscando noticias para: optimi health
Encontradas 2 noticias para optimi health
Buscando noticias para: Tilray
Encontradas 25 noticias para Tilray
Buscando noticias para: Numinus Wellness
Encontradas 3 noticias para Numinus Wellness
Buscando noticias para: MindMed
Encontradas 8 noticias para MindMed
Buscando noticias para: Compass Pathways
Encontradas 20 noticias para Compass Pathways
Buscando noticias para: Field Trip Health
Encontradas 100 noticias para Field Trip Health
Buscando noticias para: Cybin
Encontradas 4 noticias para Cybin
Buscando noticias para: ATAI Life Sciences
Encontradas 8 noticias para ATAI Life Sciences
Buscando noticias para: Seelos Therapeutics
Encontradas 2 noticias para Seelos Therapeutics
Buscando noticias para: Mind Cure Health
Encontradas 68 noticias para Mind Cure Health
Buscando noticias para: Tryp Therapeutics
Encontradas 1 noticias para Tryp Therapeutics
Buscando noticias para: 

In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for company, articles in news_data.items():
    if isinstance(articles, list):
        for article in articles:
            title = article.get('title', 'No title')
            snippet = article.get('snippet', 'No snippet')
            link = article.get('link', 'No link')
            extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Filtrar empresas sin noticias
filtered_elements = [element for element in extracted_elements if 'Title: No title'or'snippet:Nosi' not in element]

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in filtered_elements]


# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [11]:

# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = [
    "¿Cuáles son los temas más mencionados en las noticias recientes sobre la empresa?",
    "¿Qué titulares de noticias se repiten entre los diferentes artículos sobre la empresa",
    "Podrias compartirme los links de las noticas"
    "¿Qué términos o frases aparecen con mayor frecuencia en los titulares de las noticias sobre la empresa?",
    "¿Cuáles son los impactos y consecuencias más destacados mencionados en las noticias sobre la empresa?",
    "¿Qué personas o entidades son mencionadas con mayor frecuencia en las noticias sobre la empresa?",
]


# Almacenar las respuestas en una lista
responses = []

for empresa in news_data:
    print(f"Analizando preguntas para: {empresa}")
    for j, query in enumerate(questions):
        full_query = f"{query} {empresa}"
        response = qa_chain({"query": full_query})
        responses.append({
            "empresa": empresa,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {empresa}")



Analizando preguntas para: Silo Pharma
Pregunta 1/5 analizada para Silo Pharma
Pregunta 2/5 analizada para Silo Pharma
Pregunta 3/5 analizada para Silo Pharma
Pregunta 4/5 analizada para Silo Pharma
Pregunta 5/5 analizada para Silo Pharma
Analizando preguntas para: optimi health
Pregunta 1/5 analizada para optimi health
Pregunta 2/5 analizada para optimi health
Pregunta 3/5 analizada para optimi health
Pregunta 4/5 analizada para optimi health
Pregunta 5/5 analizada para optimi health
Analizando preguntas para: Tilray
Pregunta 1/5 analizada para Tilray
Pregunta 2/5 analizada para Tilray
Pregunta 3/5 analizada para Tilray
Pregunta 4/5 analizada para Tilray
Pregunta 5/5 analizada para Tilray
Analizando preguntas para: Numinus Wellness
Pregunta 1/5 analizada para Numinus Wellness
Pregunta 2/5 analizada para Numinus Wellness
Pregunta 3/5 analizada para Numinus Wellness
Pregunta 4/5 analizada para Numinus Wellness
Pregunta 5/5 analizada para Numinus Wellness
Analizando preguntas para: MindM

In [12]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(responses)

[         {         'empresa': 'Silo Pharma',
                    'query': '¿Cuáles son los temas más mencionados en las '
                             'noticias recientes sobre la empresa?',
                    'response': 'Los temas más mencionados en las noticias '
                                'recientes sobre Silo Pharma incluyen el '
                                'desarrollo de medicamentos para el Alzheimer, '
                                'el aumento significativo en el valor de sus '
                                'acciones, y una asociación con Resyca para '
                                'tecnología de entrega de medicamentos. '
                                'Además, han asegurado una licencia global '
                                'exclusiva para un tratamiento terapéutico '
                                'relacionado con el Alzheimer.'},
          {         'empresa': 'Silo Pharma',
                    'query': '¿Qué titulares de noticias se repiten entre los 

In [6]:
texto = ''' Crowe mencionan '
                                'a Vaughn Crowe, un nativo de Newark, NJ,'''

In [7]:


texto_limpio = ' '.join(texto.split())

print(texto_limpio)

Crowe mencionan ' 'a Vaughn Crowe, un nativo de Newark, NJ,


<h1> Cannabis en general <h1>


In [4]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de temas relacionados con cannabis
temas = [
    "Cannabis legalization",
    "Medical marijuana",
    "Cannabis industry",
    "Cannabis investment",
    "Cannabis stocks",
    "Cannabis research",
    "Cannabis news",
    "Cannabis policy",
    "Cannabis business",
    "Cannabis culture",
    "Psychedelics research",
    "Psychedelics industry",
    "Psychedelics investment",
    "Psychedelics news",
    "Psychedelics policy",
    "Psychedelics business",
    "Psychedelics culture"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 20 días)
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

news_data = {}

for tema in temas:
    print(f"Buscando noticias para: {tema}")
    params = {
        'q': tema,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        if articles:
            news_data[tema] = []
            for article in articles:
                title = article['title']
                link = article['url']
                snippet = article['description']
                published_at = article['publishedAt']
                news_data[tema].append(
                    {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
            print(f"Encontradas {len(articles)} noticias para {tema}")
        else:
            print(f"No se encontraron noticias para {tema}")
    except requests.exceptions.RequestException as e:
        print(f"Error al buscar noticias para {tema}: {str(e)}")

# Filtrar temas sin noticias
news_data = {tema: articles for tema, articles in news_data.items() if articles}

# No guardar en archivo JSON, continuar con el procesamiento en memoria

Buscando noticias para: Cannabis legalization
Encontradas 58 noticias para Cannabis legalization
Buscando noticias para: Medical marijuana
Encontradas 100 noticias para Medical marijuana
Buscando noticias para: Cannabis industry
Encontradas 100 noticias para Cannabis industry
Buscando noticias para: Cannabis investment
Encontradas 100 noticias para Cannabis investment
Buscando noticias para: Cannabis stocks
Encontradas 65 noticias para Cannabis stocks
Buscando noticias para: Cannabis research
Encontradas 100 noticias para Cannabis research
Buscando noticias para: Cannabis news
Encontradas 100 noticias para Cannabis news
Buscando noticias para: Cannabis policy
Encontradas 100 noticias para Cannabis policy
Buscando noticias para: Cannabis business
Encontradas 100 noticias para Cannabis business
Buscando noticias para: Cannabis culture
Encontradas 46 noticias para Cannabis culture
Buscando noticias para: Psychedelics research
Encontradas 48 noticias para Psychedelics research
Buscando not

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for tema, articles in news_data.items():
    for article in articles:
        title = article.get('title', 'No title')
        snippet = article.get('snippet', 'No snippet')
        link = article.get('link', 'No link')
        extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in extracted_elements]

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [6]:
questions = [
    "Cuales son los temas mas populares y mencionados en las noticias recientes sobre el tema, menciona los temas",
    "¿Cuáles son los temas más mencionados en las noticias recientes sobre el tema?",
    "Podrias traerme los links de esas noticias?"
    "¿Qué titulares de noticias se repiten entre los diferentes artículos sobre el tema?",
    "¿Qué términos o frases aparecen con mayor frecuencia en los titulares de las noticias sobre el tema?",
    "¿Cuáles son los impactos y consecuencias más destacados mencionados en las noticias sobre el tema?",
    "¿Qué personas o entidades son mencionadas con mayor frecuencia en las noticias sobre el tema?"
]

# Almacenar las respuestas en una lista
responses = []

for tema in news_data:
    print(f"Analizando preguntas para: {tema}")
    for j, query in enumerate(questions):
        full_query = f"{query} {tema}"
        response = qa_chain({"query": full_query})
        responses.append({
            "tema": tema,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {tema}")



Analizando preguntas para: Cannabis legalization


/home/ozz/anaconda3/envs/envopenai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Pregunta 1/6 analizada para Cannabis legalization
Pregunta 2/6 analizada para Cannabis legalization
Pregunta 3/6 analizada para Cannabis legalization
Pregunta 4/6 analizada para Cannabis legalization
Pregunta 5/6 analizada para Cannabis legalization
Pregunta 6/6 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/6 analizada para Medical marijuana
Pregunta 2/6 analizada para Medical marijuana
Pregunta 3/6 analizada para Medical marijuana
Pregunta 4/6 analizada para Medical marijuana
Pregunta 5/6 analizada para Medical marijuana
Pregunta 6/6 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/6 analizada para Cannabis industry
Pregunta 2/6 analizada para Cannabis industry
Pregunta 3/6 analizada para Cannabis industry
Pregunta 4/6 analizada para Cannabis industry
Pregunta 5/6 analizada para Cannabis industry
Pregunta 6/6 analizada para Cannabis industry
Analizando preguntas para: Cannabis investment
Pregunta 1/

In [7]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(responses)

[       {       'query': 'Cuales son los temas mas populares y mencionados en '
                         'las noticias recientes sobre el tema, menciona los '
                         'temas',
                'response': 'No tengo información específica sobre los temas '
                            'más populares y mencionados en las noticias '
                            'recientes sobre la legalización del cannabis.',
                'tema': 'Cannabis legalization'},
        {       'query': '¿Cuáles son los temas más mencionados en las '
                         'noticias recientes sobre el tema?',
                'response': 'No sé.',
                'tema': 'Cannabis legalization'},
        {       'query': 'Podrias traerme los links de esas noticias?¿Qué '
                         'titulares de noticias se repiten entre los '
                         'diferentes artículos sobre el tema?',
                'response': 'Aquí tienes los enlaces de las noticias '
                     

In [ ]:
texto = '''     **Crecimiento del mercado**: Se espera que el '
                            'mercado del cannabis alcance un valor '
                            'significativo, estimado en $75.09 mil millones '
                            'para 2029, lo que atrae la atención de los '
                            'inversores.\n',
 '''

texto_limpio = ' '.join(texto.split())

print(texto_limpio)


**Crecimiento del mercado**: Se espera que el ' 'mercado del cannabis alcance un valor ' 'significativo, estimado en $75.09 mil millones ' 'para 2029, lo que atrae la atención de los ' 'inversores. ',


In [ ]:
questions = [
    "Cuales son los temas populares y mencionados en las noticias recientes sobre el tema, menciona los temas",
    "¿Cuáles son los temas con los que se podria crear un noticia financiera sobre el tema?",
    "¿Qué temas son relevantes para la industria del tema?",
    "¿Qué temas son importantes para los inversores en el tema?",
    "¿Qué temas son relevantes para la política sobre el tema?",
    "¿Qué temas son destacados en las noticias sobre el tema?",

  
]

# Almacenar las respuestas en una lista
responses1 = []

for tema in news_data:
    print(f"Analizando preguntas para: {tema}")
    for j, query in enumerate(questions):
        full_query = f"{query} {tema}"
        response = qa_chain({"query": full_query})
        responses1.append({
            "tema": tema,
            "query": query,
            "link": link,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {tema}")

Analizando preguntas para: Cannabis legalization
Pregunta 1/6 analizada para Cannabis legalization
Pregunta 2/6 analizada para Cannabis legalization
Pregunta 3/6 analizada para Cannabis legalization
Pregunta 4/6 analizada para Cannabis legalization
Pregunta 5/6 analizada para Cannabis legalization
Pregunta 6/6 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/6 analizada para Medical marijuana
Pregunta 2/6 analizada para Medical marijuana
Pregunta 3/6 analizada para Medical marijuana
Pregunta 4/6 analizada para Medical marijuana
Pregunta 5/6 analizada para Medical marijuana
Pregunta 6/6 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/6 analizada para Cannabis industry
Pregunta 2/6 analizada para Cannabis industry
Pregunta 3/6 analizada para Cannabis industry
Pregunta 4/6 analizada para Cannabis industry
Pregunta 5/6 analizada para Cannabis industry
Pregunta 6/6 analizada para Cannabis industry
Analizand

In [ ]:
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(responses1)

[         {         'link': 'https://www.digitaljournal.com/tech-science/op-ed-psychedelics-change-your-mind-long-term-yes/article',
                    'query': 'Cuales son los temas populares y mencionados en '
                             'las noticias recientes sobre el tema, menciona '
                             'los temas',
                    'response': 'Los temas populares y mencionados en las '
                                'noticias recientes sobre la legalización del '
                                'cannabis incluyen:\n'
                                '\n'
                                '1. **Deportaciones relacionadas con el '
                                'cannabis**: Se ha informado que decenas de '
                                'miles de inmigrantes han sido deportados '
                                'debido a delitos relacionados con el '
                                'cannabis, lo que ha generado preocupación '
                                'sobre la

<h1> NuBank <h1>

In [ ]:
# Cálculo del costo total basado en las nuevas áreas y el precio por metro cuadrado

# Áreas en metros cuadrados
centro_convenciones = 100
restaurante = 50
zonas_descarga = 50
oficinas = 50

# Precios por metro cuadrado en COP
precio_minimo_m2 = 50000
precio_maximo_m2 = 60000

# Cálculo del área total
area_total = centro_convenciones + restaurante + zonas_descarga + oficinas

# Cálculo del costo total basado en el precio por metro cuadrado
costo_total_minimo = area_total * precio_minimo_m2
costo_total_maximo = area_total * precio_maximo_m2

costo_total_minimo, costo_total_maximo, area_total

(12500000, 15000000, 250)